<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/basilbot/basilbot_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# バジルさんの記事「Binanceで数か月運用していた高頻度botの紹介」を実装してみる。
https://note.com/kkngo/n/n13fb59bacc95?sub_rt=share_sb

## バックテスト　＆　フォーワードテスト

[テストネットによる本番運用はこちら](.//basilbot_real_run_testnet.ipynb)

## ロジック
バジルさんのロジックを引用すると以下のとおり

* 取引頻度
    約定履歴から作成した3秒足ごと
* エントリー/イグジット条件
    * 常に買いから入り、終値*0.9995など決め打ちでの買い指値（数字は仮）
    * イグジットも終値*1.0005など決め打ちでの売り指値（数字は仮）

* ピラミッディングやドテンの類はなし
* 手持ちのTUSD（2500TUSDくらい）分のBTCを買って売るだけ
* 4/27に大きく焼かれてからは、直近数時間の値動きが大きすぎるときはエントリーしないように対応
* 約定回数
    200～300回/日


これを今回の実装用に調整・具体化すると以下。

* 3秒毎にエントリーするかどうかを判定。

* エントリー条件
  * 前回のオーダーが残っていない（オーダー中および買いポジション(相当)がない）
  * 直前の値動きが大きくない
    * ３０分足を４本とり、平均の値幅が閾値以下。
* オーダーキャンセル条件
  * EXPIRE_TIME(600秒: 10分)以上約定しないオーダーはキャンセルする。
* 執行戦略
  * 買い
    * エントリー条件が揃ったら執行
    * 指値: 3秒足の終値 * (1-OFFSET)     // OFFSETは0.0005などの値
    * 数量: 0.001 BTC
  * 売り
    * 買いオーダーの約定が残っていたら執行
    * 指値：買いオーダー発行時の3秒足の終値 * (1+OFFSET)     // OFFSETは0.0005などの値
    * 数量：約定した買い注文と同数（=仮想敵なPosition）
    * その他：売りオーダーがExpireしてCancelされた場合、即、market_orderで投げうる。


In [1]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
#! pip install -i https://test.pypi.org/simple/ rbot
! pip install rbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://test.pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 26.9 MB/s eta 0:00:00


In [2]:
# 関連ライブラリのインストール
# Polarsは0.20.0が必要です。
! pip install pyarrow
! pip install --upgrade polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2
  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7593 sha256=a5a44f41490054011322c6328ceaa9bdb58e808246d4db3ac5a847e4bd6bc226
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html


# Agent（BOT）の実装

In [3]:
class BasilAgent:
    def __init__(self):
        self.OFFSET = 0.00_05
        self.EXPIRE_TIME = 600    # 600[sec] = 10[min]
        self.ORDER_SIZE = 0.01
        self.RANGE = 300

    def on_init(self, session):
        session.clock_interval_sec = 3     # 3秒ごとに on_clock を呼び出す

    def on_clock(self, session, clock):
        if session.expire_order(self.EXPIRE_TIME):         # 期限切れの注文をキャンセルする.
            return                                          # 期限切れがあればリターン

        if session.buy_orders or session.sell_orders:   # 既に注文がある場合はリターン
            return

        # 1時間足のレンジを計算してログに出力する。レンジが大きい場合はトレードしない。
        ohlcv1h = session.ohlcv(60*60, 4)
        range = (ohlcv1h['high']-ohlcv1h['low']).mean()
        session.log_indicator("range", range)

        if self.RANGE < range:
            return

        ohlcv = session.ohlcv(3, 1)         # 3秒足を1本分取得。
        if len(ohlcv) < 1:                 # 3秒間に約定データがない場合リターン
            print("NO OHLCV DATA")
            return

        if session.position <= 0.001:    # ポジションが少ない場合は買い注文を出す。
            order_price = ohlcv['close'][-1] * (1 - self.OFFSET)
            print("BUY ORDER: ", order_price, self.ORDER_SIZE)
            session.limit_order('Buy', order_price, self.ORDER_SIZE - session.position)

        else:                       # ポジションがある場合は売り注文を出す。
            order_price = ohlcv['close'][-1] * (1 + self.OFFSET)
            print("SELL ORDER: ", order_price, session.position)
            session.limit_order('Sell', order_price, session.position)

    def on_update(self, session, updated_order):
        # 売りオーダーが期限切れされた場合には、成り行きで売り注文を出す（ロスカット）
        if updated_order.status == 'Canceled':
            if updated_order.side == 'Sell':
                session.market_order('Sell', updated_order.remaining_size)



## バックテスト

In [4]:
from rbot import NOW, DAYS

In [5]:
# Google Colabの場合は、GoogleDriveにDBをつくります。
if 'google.colab' in str(get_ipython()):
    # Google Driveをマウントする
    from google.colab import drive
    drive.mount('/content/drive')

    # DBの保存先をGoogle Driveに変更する
    import os
    os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'    # MyDrive以下を指定しましたが適宜変更ください


Mounted at /content/drive


In [6]:
# Binanceマーケットを指定します。(BinanceかBybitのどちらかを選択してください。)
from rbot import Binance
from rbot import BinanceConfig


binance_exchange = Binance(production=True)     # 本番ネットのデータを取得します。

config = BinanceConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
binance_market = binance_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [7]:
# Bybitマーケットを指定します。(binanceかbybitのどちらか一方を選択してください)
from rbot import Bybit
from rbot import BybitConfig



bybit_exchange = Bybit(production=True)     # 本番ネットのデータを取得します。

config = BybitConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
bybit_market = bybit_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [8]:
# どちらかを選ぶ

exchange = bybit_exchange
market = bybit_market
# exchange = binance_exchange
# market = binance_market

In [9]:
# バックテストの期間を設定
BACKTEST_DAYS = 2

# バックテスト用に過去データをダウンロード
market.download_archive(
    ndays=BACKTEST_DAYS,          # 過去2日分のデータをダウンロード
    verbose=True,
)

3770143

In [10]:
from rbot import Runner
agent = BasilAgent()        # 実行対象のエージェントを指定
runner = Runner()           # 実行モジュール Runner を作成

In [11]:
session = runner.back_test(
    exchange=exchange,              # Exchangeを指定
    agent=agent,                    # エージェントを指定
    market=market,                  # マーケットを指定
    start_time=NOW()-DAYS(BACKTEST_DAYS), # 開始時刻を指定
    end_time=0,                     # 終了時刻を指定(0を指定すると最後まで実行)
    # execute_time=60*60*24,        # 実行時間を指定(指定しない、もしくは0を指定すると最後まで実行)
    verbose=True,                   # 実行の進捗を表示
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)


BUY ORDER:  65446.96015 0.01
SELL ORDER:  65476.3218 0.01
BUY ORDER:  65448.05960000001 0.01
SELL ORDER:  65462.614949999996 0.01
BUY ORDER:  65432.567350000005 0.01
BUY ORDER:  65406.580350000004 0.01
SELL ORDER:  65427.2973 0.01


In [12]:
log = session.log
orders = log.orders
orders.head(100)

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""BTCUSDT""",2024-04-03 00:00:00.747500,2024-04-03 00:00:00.599500,"""New""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Buy""","""Limit""",65446.96,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,-654.4696,0.0,654.4696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""BTCUSDT""",2024-04-03 00:00:10.169800,2024-04-03 00:00:00.599500,"""Filled""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Buy""","""Limit""",65446.96,0.01,0.0,"""BasilAgent-000…",65446.96,0.01,654.4696,0.0,"""USDT""",true,"""""",0.0,0.0,-654.4696,0.01,0.0,0.01,-654.4696,0.0,0.01,0.0,0.01,0.0,0.065447,-0.065447,-0.065447
3,"""BTCUSDT""",2024-04-03 00:00:12.101100,2024-04-03 00:00:12,"""New""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Sell""","""Limit""",65476.32,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,0.0,-0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,-0.065447
4,"""BTCUSDT""",2024-04-03 00:00:17.940400,2024-04-03 00:00:12,"""PartiallyFille…","""BasilAgent-4_q…","""BasilAgent-4_q…","""Sell""","""Limit""",65476.32,0.01,0.009,"""BasilAgent-000…",65476.32,0.001,65.47632,0.0,"""USDT""",true,"""""",0.0,0.0,65.47632,-0.001,65.47632,-0.001,0.0,-0.001,0.0,0.001,0.009,0.02936,0.006548,0.022812,-0.042635
5,"""BTCUSDT""",2024-04-03 00:00:17.981,2024-04-03 00:00:12,"""Filled""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Sell""","""Limit""",65476.32,0.01,0.0,"""BasilAgent-000…",65476.32,0.009,589.28688,0.0,"""USDT""",true,"""""",0.0,0.0,589.28688,-0.009,589.28688,-0.009,0.0,-0.009,0.0,0.009,0.0,0.26424,0.058929,0.205311,0.162677
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
13,"""BTCUSDT""",2024-04-03 00:14:15.512900,2024-04-03 00:04:12,"""Canceled""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Buy""","""Limit""",65432.57,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,654.3257,0.0,-654.3257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.177266
14,"""BTCUSDT""",2024-04-03 00:14:18.275300,2024-04-03 00:14:18,"""New""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Buy""","""Limit""",65406.58,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,-654.0658,0.0,654.0658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.177266
15,"""BTCUSDT""",2024-04-03 00:14:31.911400,2024-04-03 00:14:18,"""Filled""","""BasilAgent-4_q…","""BasilAgent-4_q…","""Buy""","""Limit""",65406.58,0.01,0.0,"""BasilAgent-000…",65406.58,0.01,654.0658,0.0,"""USDT""",true,"""""",0.0,0.0,-654.0658,0.01,0.0,0.01,-654.0658,0.0,0.01,0.0,0.01,0.0,0.065407,-0.065407,0.111859


In [13]:
# オーダー状況分析
# あまりにCancelが多い場合は、Expireの時間やオーダーのOFFSETを調整する必要がある。
orders.group_by(['order_side', 'status']).len()

order_side,status,len
str,str,u32
"""Buy""","""Filled""",3
"""Buy""","""PartiallyFille…",2
"""Sell""","""PartiallyFille…",1
"""Buy""","""New""",4
"""Sell""","""Filled""",3
"""Buy""","""Canceled""",1
"""Sell""","""New""",3


In [14]:
# 最終想定損益
orders['sum_profit'][-1]

0.25363217

In [15]:
# 手数料なし想定損益
orders['profit'].sum()

0.6462999999999999

In [16]:

# ローソク足データの作成
ohlcv = market.ohlcv(
    runner.start_timestamp,     # バックテストの開始時刻
    runner.last_timestamp,    # バックテストの終了時刻
    30                          # 30秒足
)
ohlcv.tail(5)

timestamp,open,high,low,close,volume,count
datetime[μs],f64,f64,f64,f64,f64,u32
2024-04-03 00:42:30.036900,65180.8,65209.9,65131.4,65207.0,118.404,1179
2024-04-03 00:43:00.036900,65207.0,65232.6,65104.1,65160.5,120.288,1044
2024-04-03 00:43:30.036900,65160.0,65230.8,65142.6,65230.8,44.974,521
2024-04-03 00:44:00.036900,65230.8,65241.5,65156.4,65163.1,55.429,699
2024-04-03 00:44:30.036900,65161.4,65209.7,65145.1,65187.9,50.56,613


In [17]:
# session.log_indicator('range', value)で保存したデータは以下で取得できます。戻り値はpolars.DataFrameです。
log['range'].tail(5)

timestamp,range
datetime[μs],f64
2024-04-03 00:44:45,1218.8
2024-04-03 00:44:48,1218.8
2024-04-03 00:44:51,1218.8
2024-04-03 00:44:54,1218.8
2024-04-03 00:44:57,1218.8


In [18]:
# バックテスト結果の可視化

# 数千件のデータを可視化するときは、plotlyを使うと便利。
# https://plotly.com/python/
# ただし約定データが万を超えると、ブラウザが重くなるので注意。一部データ切り取りなどが必要。

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


# フォーワードテスト

ここからは、WebSocketでリアルタイムデータを使います。そのためColabでは動きません。ローカルにjupyter環境を準備して実行してください。

In [ ]:
runner = Runner()

session = runner.dry_run(
    agent=agent,
    market=market,
    log_memory=True,
    execute_time=60*5,     # 60x5=5分間
    verbose=True,
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)

In [ ]:
log = session.log

In [ ]:
orders = log.orders
orders.tail(5)

In [ ]:
ohlcv = market.ohlcv(runner.start_timestamp, runner.last_timestamp, 10)
ohlcv

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Forwadtest Result")
